# 爬蟲程式介紹

1. 寫爬蟲程式是一種蒐集大量資料的手段，讓程式快速尋訪網站的所有內容，我們再從中擷取想要的部分。
2. 每個網站的設計不可能一樣，所以爬蟲程式的製作方式也是因網站而異。
3. 網站多半都有防護措施，防止駭客癱瘓網頁或是盜取資料。爬蟲程式因為用程式去存取網站，當然也很有可能被網站認出是機器人，然後將你封鎖。


## 範例：THE REPTILE DATABASE

以下內容以THE REPTILE DATABASE http://www.reptile-database.org/ 這個網站來當作爬蟲的目標。此網站收錄許多爬蟲類的照片及物種資料，蒐集下來可以作為訓練資料。我們示範從中撈取所有 烏龜 的照片。


首先先觀察網站，隨意搜尋一個烏龜，例如：豹紋陸龜(Stigmochelys pardalis)。觀察網頁內容、圖片呈現、網址組成。

物種的照片會集中在網頁正中間，希望用程式可以直接抓出那塊照片的HTML tag。

網址上可以看到網址參數包含屬名及種名，也就是說，修改那兩個參數就可以瀏覽不同物種的網頁。

![img01](img/img01.png)

觀察首頁，首頁左方有個Data/downloads的連結。

![img02](img/img02.png)

發現可以在這邊下載到所有物種清單，是Excel格式

![img03](img/img03.png)

通常網頁都有加javascript，js是在使用者端的瀏覽器裡執行的，可以製作一些網頁效果，或是延遲載入功能。但是如果今天是用程式來下載網頁的話，我們會下載到js的程式碼，但沒去執行js。

所以我們要進一步用F12的功能，試著關掉javascipt，看這樣的網頁中是否仍然有烏龜照片。

結果看到的網頁呈現果然不太一樣，原本中間類似相簿的內容變成了一列的圖片。

![img04](img/img04.png)

接著我們找出那列圖片部分的HTML，可以看到有一個`<div id="gallery">`的tag中，裝著很多`<a>`的tag，裏頭有很多jpg的網址，這些似乎就是我們要的照片的網址。

![img05](img/img05.png)

我們需要用程式來解析出網頁中的圖片。

在Python中常用`requests`套件送出網頁請求，`Beautiful Soup`可用來解析HTML。

我們試著以這兩個套件來取得網頁，並取出網頁中的圖片。

In [ ]:
# 缺少套件的話，用以下指令安裝套件
# !pip install requests beautifulsoup4 lxml 

In [2]:
import os
import requests
from bs4 import BeautifulSoup

In [3]:
response = requests.get('https://reptile-database.reptarium.cz/species?genus=Stigmochelys&species=pardalis')
soup = BeautifulSoup(response.text, "lxml")

In [4]:
#print(soup.prettify())

In [5]:
gallery = soup.find("div", {"id": "gallery"})
#gallery

In [6]:
imglist = gallery.find_all('a')
#imglist

In [7]:
for i in imglist[:3]:
    print(i.get('href'))

https://www.reptarium.cz/content/photo_rd_05/Stigmochelys-pardalis-03000033508_01.jpg
https://www.reptarium.cz/content/photo_rd_00/Stigmochelys-pardalis-03000027735_01.jpg
https://www.reptarium.cz/content/photo_rd_00/Stigmochelys-pardalis-03000027736_01.jpg


如此一來，我們能夠從網頁中取得所有圖片的下載連結


In [8]:
def get_species_url(genus: str, species: str) -> str:
    '''取得某物種的網址'''
    return 'https://reptile-database.reptarium.cz/species?genus={}&species={}'.format(genus, species)

In [9]:
def get_image_list(html: str) -> list:
    '''解析HTML網頁，取得網頁中物種照片的URL'''
    soup = BeautifulSoup(html, 'lxml')
    gallery = soup.find("div", {"id": "gallery"})
    imglist = gallery.find_all('a')
    imglist = [i.get('href') for i in imglist]
    return imglist

In [10]:
def download_file(url: str, path: str):
    '''給予URL和儲存路徑，將檔案下載儲存'''
    response = requests.get(url)
    if response.status_code == 200:
        with open(path, 'wb') as f:
            f.write(response.content)
    else:
        print('Download', url, 'failed:', response.status_code, response.reason)

In [11]:
def download_file_list(urls: str, directory: str, debug=False):
    '''將URL列表下載，並儲存至指定的資料夾中'''
    os.makedirs(directory, exist_ok=True)
    for i, url in enumerate(urls):
        path = os.path.join(directory, 'img{:03d}_{}'.format(i, os.path.basename(url)))
        if debug:
            print('Write path:', path)
        download_file(url, path)

In [12]:
def download_species(root_path: str, genus: str, species: str, debug=False):
    '''下載某個物種的所有照片，儲存至硬碟中'''
    if debug:
        print('Download species:', genus, species)
    url = get_species_url(genus, species)
    response = requests.get(url)
    imglist = get_image_list(response.text)
    directory = os.path.join(root_path, '{}_{}'.format(genus, species))
    download_file_list(imglist, directory, debug=debug)

In [13]:
download_species('dataset', 'Stigmochelys', 'pardalis', debug=True)

Download species: Stigmochelys pardalis
Write path: dataset/Stigmochelys_pardalis/img000_Stigmochelys-pardalis-03000033508_01.jpg
Write path: dataset/Stigmochelys_pardalis/img001_Stigmochelys-pardalis-03000027735_01.jpg
Write path: dataset/Stigmochelys_pardalis/img002_Stigmochelys-pardalis-03000027736_01.jpg
Write path: dataset/Stigmochelys_pardalis/img003_Stigmochelys-pardalis-03000028388_01.jpg
Write path: dataset/Stigmochelys_pardalis/img004_Stigmochelys-pardalis-03000028389_01.jpg
Write path: dataset/Stigmochelys_pardalis/img005_Stigmochelys-pardalis-03000028390_01.jpg
Write path: dataset/Stigmochelys_pardalis/img006_Stigmochelys-pardalis-03000035428_01.jpg
Write path: dataset/Stigmochelys_pardalis/img007_Stigmochelys-pardalis-03000048182_01.jpg
Write path: dataset/Stigmochelys_pardalis/img008_0010.jpeg
Write path: dataset/Stigmochelys_pardalis/img009_1494.jpeg
Write path: dataset/Stigmochelys_pardalis/img010_0009.jpeg
Write path: dataset/Stigmochelys_pardalis/img011_0072.jpeg
Writ

如此一來，我們能夠下載單一物種的所有圖片了，接下來如果有所有物種的清單，我們就可以爬到所有物種的照片了。

剛剛有個網頁能夠下載一個Excel檔，他正是我們要的物種清單。而在python中，我們能夠用`pandas`來讀取該Excel檔的內容。

In [14]:
# !pip install pandas

In [15]:
import pandas as pd

In [16]:
excel_url = 'http://www.reptile-database.org/data/reptile_checklist_2022_03.xlsx'
excel_name = os.path.basename(excel_url)
download_file(excel_url, excel_name)

In [17]:
excel = pd.read_excel(excel_name)
excel.head()

,type_species,Species,Author,Subspecies,Order,Family,change,sp#
0,NaN,Ablepharus alaicus,(ELPATJEVSKY 1901),Ablepharus alaicus kucenkoi NIKOLSKY 1902\nAbl...,Sauria,Scincidae,new genus (was: Asymblepharus),11256
1,NaN,Ablepharus anatolicus,SCHMIDTLER 1997,NaN,Sauria,Scincidae,NaN,23791
2,NaN,Ablepharus bivittatus,(MENETRIES 1832),NaN,Sauria,Scincidae,NaN,10001
3,NaN,Ablepharus budaki,"GÖCMEN, KUMLUTAS & TOSUNOGLU 1996",NaN,Sauria,Scincidae,NaN,10002
4,NaN,Ablepharus chernovi,DAREVSKY 1953,Ablepharus chernovi chernovi DAREVSKY 1953\nAb...,Sauria,Scincidae,NaN,10003


In [18]:
# 過濾出所有烏龜類
testudines = excel[excel['Order'] == 'Testudines']
testudines

,type_species,Species,Author,Subspecies,Order,Family,change,sp#
71,NaN,Acanthochelys macrocephala,"(RHODIN, MITTERMEIER & MCMORRIS 1984)",NaN,Testudines,Chelidae,NaN,10047
72,NaN,Acanthochelys pallidipectoris,(FREIBERG 1945),NaN,Testudines,Chelidae,NaN,10048
73,NaN,Acanthochelys radiolata,(MIKAN 1820),NaN,Testudines,Chelidae,NaN,10049
74,x,Acanthochelys spixii,(DUMÉRIL & BIBRON 1835),NaN,Testudines,Chelidae,NaN,10050
203,x,Actinemys marmorata,(BAIRD & GIRARD 1852),NaN,Testudines,Emydidae,NaN,10147
...,...,...,...,...,...,...,...,...
10887,NaN,Trachemys terrapen,(BONNATERRE 1789),NaN,Testudines,Emydidae,NaN,19328
10888,NaN,Trachemys venusta,(GRAY 1855),Trachemys venusta venusta (GRAY 1855)\nTrachem...,Testudines,Emydidae,NaN,19329
10889,NaN,Trachemys yaquia,(LEGLER & WEBB 1970),NaN,Testudines,Emydidae,NaN,19330
11145,x,Trionyx triunguis,(FORSKÅL 1775),NaN,Testudines,Trionychidae,NaN,19565


In [19]:
testudines_list = list(testudines['Species'])
print(testudines_list[:10])
print('Count', len(testudines_list))

['Acanthochelys macrocephala', 'Acanthochelys pallidipectoris', 'Acanthochelys radiolata', 'Acanthochelys spixii', 'Actinemys marmorata', 'Actinemys pallida', 'Aldabrachelys gigantea', 'Amyda cartilaginea', 'Amyda ornata', 'Apalone ferox']
Count 356


如此一來，我們可以從這個資料庫中過濾出所有烏龜類的學名，一共有356種。我們只要用for-loop去下載這356個物種，就可以載到很多照片了。

In [20]:
# root_path = 'dataset'
# for name in testudines_list:
#     split = name.split(' ')
#     genus = split[0]
#     species = split[1]
#     download_species(root_path, genus, species, debug=True)

下載完成的話，我們把每個物種的照片放在各自的資料夾中，資料夾的名稱就是該物種的學名。

對之後要訓練分類模型來說很方便，以資料夾的方式區別每個類別，資料夾名稱是該類別的類別名稱。

```text
dataset
├── Acanthochelys_pallidipectoris
│   └── img000_Acanthochelys-pallidipectoris-03000037284_01.jpg
├── Acanthochelys_radiolata
│   └── img000_Acanthochelys-radiolata-03000039563_01.jpg
├── Acanthochelys_spixii
│   ├── img000_Acanthochelys-spixii-03000034151_01.jpg
│   ├── img001_Acanthochelys-spixii-03000034150_01.jpg
│   └── img002_Acanthochelys-spixii-03000042193_01.jpg
├── Amyda_cartilaginea
│   ├── img000_Amyda-cartilaginea-03000029644_01.jpg
│   └── img001_Amyda-cartilaginea-03000038612_01.jpg
├── Apalone_ferox
│   ├── img000_Apalone-ferox-03000029679_01.jpg
│   ├── img001_Apalone-ferox-03000029680_01.jpg
│   └── img002_Apalone-ferox-03000045330_01.jpg
```